<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Warto%C5%9Bci_odstaj%C4%85ce/colabs/Namierzanie_outlayersow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1561, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 1561 (delta 176), reused 161 (delta 107), pack-reused 1307
Receiving objects: 100% (1561/1561), 5.99 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (945/945), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Wartości_odstające

Branch 'Wartości_odstające' set up to track remote branch 'Wartości_odstające' from 'origin'.
Switched to a new branch 'Wartości_odstające'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=030b9cbf37c65f7871a387861c3ec4f8070d71970f326271150748eff7fb379e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-14 13:49:27--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [5]:
from lib.pyspark_init import create_spark_context, add_kaggle_data
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

In [6]:
spark = create_spark_context()
spark

In [7]:
%%time
data = spark.read.parquet(JOINED_DATA)
data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-------------------+--------------------+
|   nconst|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|        primaryName|      knownForTitles|
+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+-------------------+--------------------+
|nm0000034|[tt0032636, tt003...|[movie, movie, mo...|[Invisible Stripe...|[0, 0, 0, 0, 0, 0...|     1939|     \N|[Action, Crime, D...|   actor|[["Tim Taylor"], ...|     William Holden|tt0043014,tt00463...|
|nm0000068|[tt0038613, tt002...|[movie, movie, mo...|[Home, Sweet Homi...|[0, 0, 0, 0, 0, 0...|     1931|     \N|[Adventure, Comed...|   actor|[["Lt. Bill Smith...|     Randolp

In [10]:
spark.sql("""SELECT* from data""").show()

AnalysisException: ignored